# Loading modules and packages

In [ ]:
# core
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# pre-processing
from sklearn.preprocessing import MinMaxScaler

# modelling
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU
from tensorflow.keras.optimizers import Adam

# callbacks
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# model evaluation
import tensorflow.keras.backend as K
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.utils import plot_model

# Functions used in this notebook

In [ ]:
# function to engineer features using available information from a dataframe df
def engineer_features(df):
    df['dow']  = df.date_time.dt.day_of_week # day of week
    df['woy']  = df.date_time.dt.isocalendar().week.astype('int') # week of the year
    df['hod'] = df.date_time.dt.hour # hour of the day
    df['working_hours'] = df.date_time.dt.hour.isin(np.arange(8, 19)).astype('int') # indicator of whether the record was made during working hours
    return df

# function to calculate RMSLE loss directly
def rmsle_(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred)) 

# Loading the data

In [ ]:
train       = pd.read_csv(filepath_or_buffer = '../input/tabular-playground-series-jul-2021/train.csv', parse_dates = ['date_time'])
test        = pd.read_csv(filepath_or_buffer = '../input/tabular-playground-series-jul-2021/test.csv', parse_dates = ['date_time'])
submission  = pd.read_csv(filepath_or_buffer = '../input/tabular-playground-series-jul-2021/sample_submission.csv')

# Feature engineering

In [ ]:
# adding engineered features to their respective dataframes
train = engineer_features(train)
test  = engineer_features(test)

In [ ]:
# getting the target column names
target_columns = [column for column in train.columns if column.startswith('target_')]

# getting the feature column names
feature_columns = [column for column in train.columns if column not in ['date_time'] + target_columns]

# Correlations

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols = 2, figsize=(25, 8)) 
sns.heatmap(train.drop(columns = ['date_time']).corr(), cmap = 'RdBu', ax = ax1)
sns.heatmap(test.drop(columns = ['date_time']).corr(), cmap = 'RdBu', ax = ax2)
ax1.set_title('Correlations on the training set', fontdict = {'size': 14, 'fontweight': 'bold'})
ax2.set_title('Correlations on the test set', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.show()

# Data preparation

## For the simplest form of the MLP - MLP #1

In [ ]:
# copying the original dataframe
df_mlp_1 = train.copy()

# splitting the targets from the features
X_mlp_1, y_mlp_1 = df_mlp_1[feature_columns], df_mlp_1[target_columns]

# getting the index of the sensor data and the other features
sensor_index = [column_index for column_index, column_name in enumerate(X_mlp_1.columns) if column_name.startswith('sensor')]
other_index = [column_index for column_index in range(X_mlp_1.shape[1]) if column_index not in sensor_index]

# log transforming the target columns
y_mlp_1 = np.log1p(y_mlp_1)

# instantiating the input scaler
scaler = MinMaxScaler()

# training the scaler
scaler.fit(X_mlp_1)

# applying the scaler to the data
X_mlp_1 = scaler.transform(X_mlp_1)
X_test  = scaler.transform(test[feature_columns])

## For the multi-output MLP - MLP #2

In [ ]:
# copying the dataframe from mlp_1 dataframe
X_mlp_2 = X_mlp_1.copy()

## separating each of the targets
y_CO, y_BE, y_NO = y_mlp_1.target_carbon_monoxide, y_mlp_1.target_benzene, y_mlp_1.target_nitrogen_oxides

## For the multi-feature-input single-output MLP - MLP #3

In [ ]:
# copying the dataframe from mlp_1 dataframe
X_mlp_3 = X_mlp_1.copy()

# getting the indexes for the sensor data
X_mlp_3_sensor, X_mlp_3_others = X_mlp_3[:, sensor_index], X_mlp_3[:, other_index]

## For multi-input (autoregressive targets + features) and multi-output MLP - MLP #4

In [ ]:
## function to split each of the individual sequences for the autoregressive mlp
def split_sequences_mlp_4(sequence, n_lagged_inputs):
    # creating empty lists to store the lagged inputs and the target output
    lagged_inputs, target_output = list(), list()
    # looping over the sequence
    for observation in range(len(sequence)):
        # defining the last index of this loop step
        last_idx = observation + n_lagged_inputs
        # breaking the loop if the index is beyond the number of observations
        if last_idx > len(sequence) - 1:
            break
        # extracting the inputs and output for this loop step
        inputs, output = sequence[observation:last_idx], sequence[last_idx]
        # appending the input and output sequences
        lagged_inputs.append(inputs), target_output.append(output)
    # returning the inputs and outputs
    return np.array(lagged_inputs), np.array(target_output)

In [ ]:
## extracting the splitted sequences for each of the response variables
n_ar_inputs = 6
X_ar_CO_mlp_4, y_ar_CO_mlp_4 = split_sequences_mlp_4(sequence = y_CO, n_lagged_inputs = n_ar_inputs)
X_ar_BE_mlp_4, y_ar_BE_mlp_4 = split_sequences_mlp_4(sequence = y_BE, n_lagged_inputs = n_ar_inputs)
X_ar_NO_mlp_4, y_ar_NO_mlp_4 = split_sequences_mlp_4(sequence = y_NO, n_lagged_inputs = n_ar_inputs)

## copying the original data
X_mlp_4_sensor, X_mlp_4_others = X_mlp_3_sensor.copy(), X_mlp_3_others.copy()

## removing the first 6 rows of data, since these were used to create the autoregressive features
X_mlp_4_sensor, X_mlp_4_others = X_mlp_4_sensor[n_ar_inputs:, :], X_mlp_4_others[n_ar_inputs:, :]

## For multi-input (autoregressive targets + features) and single output MLP - MLP #5

In [ ]:
## merging all target in a single 
y_ar_mlp_5 = np.column_stack([y_ar_CO_mlp_4, y_ar_BE_mlp_4, y_ar_NO_mlp_4])

## Multi-input (AR targets 6hrs+ AR temperature 12hrs + AR relative humidity 12hrs) and single output MLP - MLP #6

In [ ]:
## function to split each of the individual sequences for the autoregressive mlp
def split_feature_sequences(sequence, n_lagged_inputs):
    # creating empty lists to store the lagged inputs and the target output
    lagged_inputs = list()
    # looping over the sequence
    for observation in range(len(sequence)):
        # defining the last index of this loop step
        last_idx = observation + n_lagged_inputs
        # breaking the loop if the index is beyond the number of observations
        if last_idx > len(sequence):
            break
        # extracting the inputs and output for this loop step
        inputs = sequence[observation:last_idx]
        # appending the input and output sequences
        lagged_inputs.append(inputs)
    # returning the inputs and outputs
    return np.array(lagged_inputs)

In [ ]:
## getting the temperature and relative humidity data
X_temp_mlp_6, X_rl_mlp_6 = X_mlp_1[:, 0], X_mlp_1[:, 1]

## creating the lagged inputs for temperatura and relative humidity data
n_ar_mlp_6 = 12
X_temp_mlp_6 = split_feature_sequences(sequence = X_temp_mlp_6, n_lagged_inputs = n_ar_mlp_6)
X_rl_mlp_6   = split_feature_sequences(sequence = X_rl_mlp_6, n_lagged_inputs = n_ar_mlp_6)

## removing the data for the n_ar_mlp_6 observations from the sensor data
X_mlp_6_sensor = X_mlp_3_sensor.copy()
X_mlp_6_sensor = X_mlp_6_sensor[n_ar_mlp_6 - 1:, :]

## removing the first 6 elements of the autoregressive input objects
X_ar_CO_mlp_6, X_ar_BE_mlp_6, X_ar_NO_mlp_6 = X_ar_CO_mlp_4.copy(), X_ar_BE_mlp_4.copy(), X_ar_NO_mlp_4.copy()
X_ar_CO_mlp_6, X_ar_BE_mlp_6, X_ar_NO_mlp_6 = X_ar_CO_mlp_6[5:, :], X_ar_BE_mlp_6[5:, :], X_ar_NO_mlp_6[5:, :]

## removing the first 6 elements from the target frame so that they are all aligned
y_ar_mlp_6 = y_ar_mlp_5.copy()
y_ar_mlp_6 = y_ar_mlp_6[5:, :]

In [ ]:
# creating the test set autoregressive features
## seeding the initial list
test_ar_TEMP, test_ar_RL = X_mlp_1[-12:-1, 0].tolist(), X_mlp_1[-12:-1, 1].tolist()

## adding the rest of the time series
test_ar_TEMP, test_ar_RL = test_ar_TEMP + X_test[:, 0].tolist(), test_ar_RL + X_test[:, 1].tolist()

## creating the lagged feature sequences on the test set
test_ar_TEMP = split_feature_sequences(sequence = test_ar_TEMP, n_lagged_inputs = n_ar_mlp_6)
test_ar_RL   = split_feature_sequences(sequence = test_ar_RL, n_lagged_inputs = n_ar_mlp_6)

## Multi-input (AR targets 12hrs+ AR temperature 12hrs + AR relative humidity 12hrs) and single output MLP - MLP #7

In [ ]:
## extracting the splitted sequences for each of the response variables
n_ar_inputs_mlp_7 = 25
X_ar_CO_mlp_7, y_ar_CO_mlp_7 = split_sequences_mlp_4(sequence = y_CO, n_lagged_inputs = n_ar_inputs_mlp_7)
X_ar_BE_mlp_7, y_ar_BE_mlp_7 = split_sequences_mlp_4(sequence = y_BE, n_lagged_inputs = n_ar_inputs_mlp_7)
X_ar_NO_mlp_7, y_ar_NO_mlp_7 = split_sequences_mlp_4(sequence = y_NO, n_lagged_inputs = n_ar_inputs_mlp_7)

## merging the targets in a single array
y_ar_mlp_7 = np.column_stack([y_ar_CO_mlp_7, y_ar_BE_mlp_7, y_ar_NO_mlp_7])

## removing the data for the n_ar_inputs_mlp_7 observations from the sensor data
X_mlp_7_sensor = X_mlp_3_sensor.copy()
X_mlp_7_sensor = X_mlp_7_sensor[n_ar_inputs_mlp_7:, :]

## removing the first instance as it gets the historical data from t0 to t12, and we needed it from t1 to t13
X_temp_mlp_7 = X_temp_mlp_6.copy()
X_temp_mlp_7 = X_temp_mlp_7[14:, :]

## removing the first instance as it gets the historical data from t0 to t12, and we needed it from t1 to t13
X_rl_mlp_7 = X_rl_mlp_6.copy()
X_rl_mlp_7 = X_rl_mlp_7[14:, :]

# Modelling

## MLP #1 - Simple MLP

In [ ]:
## creating the layers
# input layer
input_layer = Input(shape = (X_mlp_1.shape[1], ))
# hidden layers
hidden_layer = Dense(units = 32, activation = 'relu')(input_layer)
hidden_layer = Dropout(0.2)(hidden_layer)
hidden_layer = Dense(units = 32, activation = 'relu')(hidden_layer)
# output layers
output_layer = Dense(units = 3)(hidden_layer)

## creating the model
model = Model(inputs = input_layer, outputs = output_layer)

# compilling the model
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 20, min_delta = 0.01, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_root_mean_squared_error', factor = 0.2, patience = 5)

# fitting the model
history = model.fit(x = X_mlp_1, y = y_mlp_1, validation_split = 0.3, batch_size = 16, epochs = 100, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_root_mean_squared_error'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(model, show_shapes = True)

In [ ]:
## extracting the predictions on the test set
test_preds = model.predict(X_test)

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_1 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_1[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_1[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_1_predictions = model.predict(X_mlp_1)

## putting the predictions in a dataframe
mlp_1_predictions = pd.DataFrame(data = mlp_1_predictions, columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(y_mlp_1.loc[:, target] - mlp_1_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(y_mlp_1.loc[:, target] - mlp_1_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

## MLP #2 - Multi-output MLP

In [ ]:
## creating the layers
# input layer
input_layer = Input(shape = (X_mlp_2.shape[1],))
# hidden layers
hidden_layer = Dense(units = 16, activation = 'relu')(input_layer)
hidden_layer = Dense(units = 32, activation = 'relu')(hidden_layer)
# output layers
output_CO = Dense(units = 1, name = 'out_CO')(hidden_layer)
output_BE = Dense(units = 1, name = 'out_BE')(hidden_layer)
output_NO = Dense(units = 1, name = 'out_NO')(hidden_layer)

## creating the model
mlp_2 = Model(inputs = input_layer, outputs = [output_CO, output_BE, output_NO])

# compilling the model
mlp_2.compile(optimizer = Adam(learning_rate = 0.001), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_loss', patience = 20, min_delta = 0.01, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5)

# fitting the model
history = mlp_2.fit(x = X_mlp_2, y = [y_CO, y_BE, y_NO], validation_split = 0.3, batch_size = 8, epochs = 400, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_loss'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'MSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

# training history for the targets
plt.figure(figsize = (10, 6))
plt.plot(history.history['val_out_CO_root_mean_squared_error'], label = 'CO')
plt.plot(history.history['val_out_BE_root_mean_squared_error'], label = 'Benzene')
plt.plot(history.history['val_out_NO_root_mean_squared_error'], label = 'NO')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(mlp_2, show_shapes = True)

In [ ]:
## extracting the predictions on the test set
test_preds = mlp_2.predict(X_test)

## reshaping the predictions
test_preds = np.column_stack(test_preds)

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_2 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_2[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_2[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_2_predictions = mlp_2.predict(X_mlp_2)

## reshaping the predictions
mlp_2_predictions = np.column_stack(mlp_2_predictions)

## putting the predictions in a dataframe
mlp_2_predictions = pd.DataFrame(data = mlp_2_predictions, columns = target_columns)

## getting the observed values
observed_values = pd.DataFrame(data = np.column_stack([y_CO, y_BE, y_NO]), columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(observed_values.loc[:, target] - mlp_2_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(observed_values.loc[:, target] - mlp_2_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

## MLP #3 - Multi-input MLP

In [ ]:
### creating the layers
## SENSOR DATA
# input layer
input_layer_1 = Input(shape = (X_mlp_3_sensor.shape[1], ))
# hidden layers
hidden_layer_1 = Dense(units = 32, activation = 'relu')(input_layer_1)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = Dense(units = 16, activation = 'relu')(hidden_layer_1)

## OTHER FEATURES
# input layer
input_layer_2 = Input(shape = (X_mlp_3_others.shape[1], ))
# hidden layers
hidden_layer_2 = Dense(units = 32, activation = 'relu')(input_layer_2)

## MERGING THE TWO INPUTS
merge_layer = Concatenate()([hidden_layer_1, hidden_layer_2])
## one more layer
hidden_layer = Dense(units = 32, activation = 'relu')(merge_layer)
hidden_layer = Dropout(0.1)(hidden_layer)
# output layers
output_layer = Dense(units = 3)(hidden_layer)

## creating the model
mlp_3 = Model(inputs = [input_layer_1, input_layer_2], outputs = output_layer)

# compilling the model
mlp_3.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 20, min_delta = 0.002, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_root_mean_squared_error', factor = 0.2, patience = 5)

# fitting the model
history = mlp_3.fit(x = [X_mlp_3_sensor, X_mlp_3_others], y = y_mlp_1, validation_split = 0.3, batch_size = 16, epochs = 100, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_root_mean_squared_error'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(mlp_3, show_shapes = True)

In [ ]:
## extracting the predictions on the test set
test_preds = mlp_3.predict([X_test[:, sensor_index], X_test[:, other_index]])

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_3 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_3[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_3[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_3_predictions = mlp_3.predict([X_mlp_3_sensor, X_mlp_3_others])

## putting the predictions in a dataframe
mlp_3_predictions = pd.DataFrame(data = mlp_3_predictions, columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(y_mlp_1.loc[:, target] - mlp_3_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(y_mlp_1.loc[:, target] - mlp_3_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

## MLP #4 - Multi-input-output autoregressive MLP 

In [ ]:
### creating the layers
## SENSOR DATA
# input layer
input_layer_1 = Input(shape = (X_mlp_4_sensor.shape[1], ))
# hidden layers
hidden_layer_1 = Dense(units = 32, activation = 'relu')(input_layer_1)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = Dense(units = 16, activation = 'relu')(hidden_layer_1)

## OTHER FEATURES
# input layer
input_layer_2 = Input(shape = (X_mlp_4_others.shape[1], ))
# hidden layers
hidden_layer_2 = Dense(units = 32, activation = 'relu')(input_layer_2)

## AUTOREGRESSIVE CO
# input layer
input_layer_CO = Input(shape = (X_ar_CO_mlp_4.shape[1], ))
# hidden layers
hidden_layer_CO = Dense(units = 32, activation = 'relu')(input_layer_CO)

## AUTOREGRESSIVE BE
# input layer
input_layer_BE = Input(shape = (X_ar_BE_mlp_4.shape[1], ))
# hidden layers
hidden_layer_BE = Dense(units = 32, activation = 'relu')(input_layer_BE)

## AUTOREGRESSIVE NO
# input layer
input_layer_NO = Input(shape = (X_ar_NO_mlp_4.shape[1], ))
# hidden layers
hidden_layer_NO = Dense(units = 32, activation = 'relu')(input_layer_NO)

## MERGING THE TWO INPUTS
merge_layer = Concatenate()([hidden_layer_1, hidden_layer_2, hidden_layer_CO, hidden_layer_BE, hidden_layer_NO])
## one more layer
hidden_layer = Dense(units = 32, activation = 'relu')(merge_layer)
hidden_layer = Dropout(0.1)(hidden_layer)
# output layers
# output layers
output_CO = Dense(units = 1, name = 'out_CO')(hidden_layer)
output_BE = Dense(units = 1, name = 'out_BE')(hidden_layer)
output_NO = Dense(units = 1, name = 'out_NO')(hidden_layer)

## creating the model
mlp_4 = Model(inputs = [input_layer_1, input_layer_2, input_layer_CO, input_layer_BE, input_layer_NO], outputs = [output_CO, output_BE, output_NO])

# compilling the model
mlp_4.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_loss', patience = 20, min_delta = 0.002, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 5)

# fitting the model
history = mlp_4.fit(x = [X_mlp_4_sensor, X_mlp_4_others, X_ar_CO_mlp_4, X_ar_BE_mlp_4, X_ar_NO_mlp_4], 
                    y = [y_ar_CO_mlp_4, y_ar_BE_mlp_4, y_ar_NO_mlp_4], 
                    validation_split = 0.3, batch_size = 8, epochs = 100, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_loss'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(mlp_4, show_shapes = True)

In [ ]:
## seeding the model prediction with the last six observations in the train set
test_seed_CO, test_seed_BE, test_seed_NO = y_CO[-7:-1].tolist(), y_BE[-7:-1].tolist(), y_NO[-7:-1].tolist()

## looping over each of the rows of the test set and generating the predictions
for instance in range(X_test.shape[0]):
    ## parsing the values to a numpy array
    test_ar_CO, test_ar_BE, test_ar_NO = np.array(test_seed_CO[-6:]), np.array(test_seed_BE[-6:]), np.array(test_seed_NO[-6:])

    ## extracting the predictions on the test set
    test_preds = mlp_4.predict([
        X_test[instance, sensor_index].reshape(1, 5),
        X_test[instance, other_index].reshape(1, 7),
        test_ar_CO.reshape(1, n_ar_inputs),
        test_ar_BE.reshape(1, n_ar_inputs),
        test_ar_NO.reshape(1, n_ar_inputs)
    ]
    )
    
    ## reshaping the predictions
    test_preds = np.reshape(test_preds, (-1, ))
    
    ## adding the predictions back to the original lists
    test_seed_CO.append(test_preds[0]), test_seed_BE.append(test_preds[1]), test_seed_NO.append(test_preds[2])
    
    ## printing the progress
    if instance % 200 == 0:
        print(f'Calculating the predictions for the time step {instance}.')

In [ ]:
## stacking the predictions side by side
test_preds = np.column_stack([test_seed_CO, test_seed_BE, test_seed_NO])

## removing the first six observation since they come from the train set
test_preds = test_preds[6:, :]

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_4 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_4[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_4[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_4_predictions = mlp_4.predict([X_mlp_4_sensor, X_mlp_4_others, X_ar_CO_mlp_4, X_ar_BE_mlp_4, X_ar_NO_mlp_4])

## reshaping the predictions
mlp_4_predictions = np.column_stack(mlp_4_predictions)

## putting the predictions in a dataframe
mlp_4_predictions = pd.DataFrame(data = mlp_4_predictions, columns = target_columns)

## getting the observed values
observed_values = pd.DataFrame(data = np.column_stack([y_ar_CO_mlp_4, y_ar_BE_mlp_4, y_ar_NO_mlp_4]), columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(observed_values.loc[:, target] - mlp_4_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(observed_values.loc[:, target] - mlp_4_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

## MLP #6 - Multi-input single-output MLP on autoregressive targets

In [ ]:
### creating the layers
## SENSOR DATA
# input layer
input_layer_1 = Input(shape = (X_mlp_4_sensor.shape[1], ))
# hidden layers
hidden_layer_1 = Dense(units = 32, activation = 'relu')(input_layer_1)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = Dense(units = 16, activation = 'relu')(hidden_layer_1)

## OTHER FEATURES
# input layer
input_layer_2 = Input(shape = (X_mlp_4_others.shape[1], ))
# hidden layers
hidden_layer_2 = Dense(units = 32, activation = 'relu')(input_layer_2)

## AUTOREGRESSIVE CO
# input layer
input_layer_CO = Input(shape = (X_ar_CO_mlp_4.shape[1], ))
# hidden layers
hidden_layer_CO = Dense(units = 32, activation = 'relu')(input_layer_CO)

## AUTOREGRESSIVE BE
# input layer
input_layer_BE = Input(shape = (X_ar_BE_mlp_4.shape[1], ))
# hidden layers
hidden_layer_BE = Dense(units = 32, activation = 'relu')(input_layer_BE)

## AUTOREGRESSIVE NO
# input layer
input_layer_NO = Input(shape = (X_ar_NO_mlp_4.shape[1], ))
# hidden layers
hidden_layer_NO = Dense(units = 32, activation = 'relu')(input_layer_NO)

## MERGING THE TWO INPUTS
merge_layer = Concatenate()([hidden_layer_1, hidden_layer_2, hidden_layer_CO, hidden_layer_BE, hidden_layer_NO])
## one more layer
hidden_layer = Dense(units = 32, activation = 'relu')(merge_layer)
hidden_layer = Dropout(0.1)(hidden_layer)
# output layers
output_layer = Dense(units = 3)(hidden_layer)

## creating the model
mlp_5 = Model(inputs = [input_layer_1, input_layer_2, input_layer_CO, input_layer_BE, input_layer_NO], outputs = output_layer)

# compilling the model
mlp_5.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 20, min_delta = 0.002, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_root_mean_squared_error', factor = 0.2, patience = 5)

# fitting the model
history = mlp_5.fit(x = [X_mlp_4_sensor, X_mlp_4_others, X_ar_CO_mlp_4, X_ar_BE_mlp_4, X_ar_NO_mlp_4], 
                    y = y_ar_mlp_5, 
                    validation_split = 0.3, batch_size = 8, epochs = 100, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_loss'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(mlp_5, show_shapes = True)

In [ ]:
## seeding the model prediction with the last six observations in the train set
test_seed_CO, test_seed_BE, test_seed_NO = y_CO[-7:-1].tolist(), y_BE[-7:-1].tolist(), y_NO[-7:-1].tolist()

## looping over each of the rows of the test set and generating the predictions
for instance in range(X_test.shape[0]):
    ## parsing the values to a numpy array
    test_ar_CO, test_ar_BE, test_ar_NO = np.array(test_seed_CO[-6:]), np.array(test_seed_BE[-6:]), np.array(test_seed_NO[-6:])

    ## extracting the predictions on the test set
    test_preds = mlp_5.predict([
        X_test[instance, sensor_index].reshape(1, 5),
        X_test[instance, other_index].reshape(1, 7),
        test_ar_CO.reshape(1, n_ar_inputs),
        test_ar_BE.reshape(1, n_ar_inputs),
        test_ar_NO.reshape(1, n_ar_inputs)
    ]
    )
    
    ## reshaping the predictions
    test_preds = np.reshape(test_preds, (-1, ))
    
    ## adding the predictions back to the original lists
    test_seed_CO.append(test_preds[0]), test_seed_BE.append(test_preds[1]), test_seed_NO.append(test_preds[2])
    
    ## printing the progress
    if instance % 200 == 0:
        print(f'Calculating the predictions for the time step {instance}.')

In [ ]:
## stacking the predictions side by side
test_preds = np.column_stack([test_seed_CO, test_seed_BE, test_seed_NO])

## removing the first six observation since they come from the train set
test_preds = test_preds[6:, :]

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_5 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_5[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_5[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_5_predictions = mlp_5.predict([X_mlp_4_sensor, X_mlp_4_others, X_ar_CO_mlp_4, X_ar_BE_mlp_4, X_ar_NO_mlp_4])

## putting the predictions in a dataframe
mlp_5_predictions = pd.DataFrame(data = mlp_5_predictions, columns = target_columns)

## getting the observed values
observed_values = pd.DataFrame(data = y_ar_mlp_5, columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(observed_values.loc[:, target] - mlp_5_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(observed_values.loc[:, target] - mlp_5_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

## MLP #6 - Multi-AR inputs (12h lag features + 6h lag targets) and single output MLP 

In [ ]:
### creating the layers
## SENSOR DATA
# input layer
input_layer_1 = Input(shape = (X_mlp_6_sensor.shape[1], ))
# hidden layers
hidden_layer_1 = Dense(units = 32, activation = 'relu')(input_layer_1)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = Dense(units = 16, activation = 'relu')(hidden_layer_1)

## TEMPERATURE DATA
# input layer
input_layer_2 = Input(shape = (X_temp_mlp_6.shape[1], ))
# hidden layers
hidden_layer_2 = Dense(units = 32, activation = 'relu')(input_layer_2)

## RELATIVE HUMIDITY DATA
# input layer
input_layer_3 = Input(shape = (X_rl_mlp_6.shape[1], ))
# hidden layers
hidden_layer_3 = Dense(units = 32, activation = 'relu')(input_layer_3)

## AUTOREGRESSIVE CO
# input layer
input_layer_CO = Input(shape = (X_ar_CO_mlp_6.shape[1], ))
# hidden layers
hidden_layer_CO = Dense(units = 32, activation = 'relu')(input_layer_CO)

## AUTOREGRESSIVE BE
# input layer
input_layer_BE = Input(shape = (X_ar_BE_mlp_6.shape[1], ))
# hidden layers
hidden_layer_BE = Dense(units = 32, activation = 'relu')(input_layer_BE)

## AUTOREGRESSIVE NO
# input layer
input_layer_NO = Input(shape = (X_ar_NO_mlp_6.shape[1], ))
# hidden layers
hidden_layer_NO = Dense(units = 32, activation = 'relu')(input_layer_NO)

## MERGING THE TWO INPUTS
merge_layer = Concatenate()([hidden_layer_1, hidden_layer_2, hidden_layer_3, hidden_layer_CO, hidden_layer_BE, hidden_layer_NO])
## one more layer
hidden_layer = Dense(units = 32, activation = 'relu')(merge_layer)
hidden_layer = Dropout(0.1)(hidden_layer)
# output layers
output_layer = Dense(units = 3)(hidden_layer)

## creating the model
mlp_6 = Model(inputs = [input_layer_1, input_layer_2, input_layer_3, input_layer_CO, input_layer_BE, input_layer_NO], outputs = output_layer)

# compilling the model
mlp_6.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 20, min_delta = 0.002, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_root_mean_squared_error', factor = 0.2, patience = 5)

# fitting the model
history = mlp_6.fit(x = [X_mlp_6_sensor, X_temp_mlp_6, X_rl_mlp_6, X_ar_CO_mlp_6, X_ar_BE_mlp_6, X_ar_NO_mlp_6], 
                    y = y_ar_mlp_6, 
                    validation_split = 0.3, batch_size = 8, epochs = 100, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_loss'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(mlp_6, show_shapes = True)

In [ ]:
## seeding the model prediction with the last six observations in the train set
test_seed_CO, test_seed_BE, test_seed_NO = y_CO[-7:-1].tolist(), y_BE[-7:-1].tolist(), y_NO[-7:-1].tolist()

## looping over each of the rows of the test set and generating the predictions
for instance in range(X_test.shape[0]):
    ## parsing the values to a numpy array
    test_ar_CO, test_ar_BE, test_ar_NO = np.array(test_seed_CO[-6:]), np.array(test_seed_BE[-6:]), np.array(test_seed_NO[-6:]), 

    ## extracting the predictions on the test set
    test_preds = mlp_6.predict([
        X_test[instance, sensor_index].reshape(1, X_mlp_6_sensor.shape[1]),
        test_ar_TEMP[instance, :].reshape(1, X_temp_mlp_6.shape[1]),
        test_ar_RL[instance, :].reshape(1, X_rl_mlp_6.shape[1]),
        test_ar_CO.reshape(1, X_ar_CO_mlp_6.shape[1]),
        test_ar_BE.reshape(1, X_ar_BE_mlp_6.shape[1]),
        test_ar_NO.reshape(1, X_ar_NO_mlp_6.shape[1])
    ]
    )
    
    ## reshaping the predictions
    test_preds = np.reshape(test_preds, (-1, ))
    
    ## adding the predictions back to the original lists
    test_seed_CO.append(test_preds[0]), test_seed_BE.append(test_preds[1]), test_seed_NO.append(test_preds[2])
    
    ## printing the progress
    if instance % 200 == 0:
        print(f'Calculating the predictions for the time step {instance}.')

In [ ]:
## stacking the predictions side by side
test_preds = np.column_stack([test_seed_CO, test_seed_BE, test_seed_NO])

## removing the first six observation since they come from the train set
test_preds = test_preds[6:, :]

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_6 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_6[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_6[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_6_predictions = mlp_6.predict([X_mlp_6_sensor, X_temp_mlp_6, X_rl_mlp_6, X_ar_CO_mlp_6, X_ar_BE_mlp_6, X_ar_NO_mlp_6])

## putting the predictions in a dataframe
mlp_6_predictions = pd.DataFrame(data = mlp_6_predictions, columns = target_columns)

## getting the observed values
observed_values = pd.DataFrame(data = y_ar_mlp_6, columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(observed_values.loc[:, target] - mlp_6_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(observed_values.loc[:, target] - mlp_6_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

## MLP #7 - Multi-AR inputs (12h lag features + 12h lag targets) and single output MLP

In [ ]:
### creating the layers
## SENSOR DATA
# input layer
input_layer_1 = Input(shape = (X_mlp_7_sensor.shape[1], ))
# hidden layers
hidden_layer_1 = Dense(units = 32, activation = 'relu')(input_layer_1)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = Dense(units = 16, activation = 'relu')(hidden_layer_1)

## TEMPERATURE DATA
# input layer
input_layer_2 = Input(shape = (X_temp_mlp_7.shape[1], ))
# hidden layers
hidden_layer_2 = Dense(units = 32, activation = 'relu')(input_layer_2)

## RELATIVE HUMIDITY DATA
# input layer
input_layer_3 = Input(shape = (X_rl_mlp_7.shape[1], ))
# hidden layers
hidden_layer_3 = Dense(units = 32, activation = 'relu')(input_layer_3)

## AUTOREGRESSIVE CO
# input layer
input_layer_CO = Input(shape = (X_ar_CO_mlp_7.shape[1], ))
# hidden layers
hidden_layer_CO = Dense(units = 64, activation = 'relu')(input_layer_CO)

## AUTOREGRESSIVE BE
# input layer
input_layer_BE = Input(shape = (X_ar_BE_mlp_7.shape[1], ))
# hidden layers
hidden_layer_BE = Dense(units = 32, activation = 'relu')(input_layer_BE)

## AUTOREGRESSIVE NO
# input layer
input_layer_NO = Input(shape = (X_ar_NO_mlp_7.shape[1], ))
# hidden layers
hidden_layer_NO = Dense(units = 32, activation = 'relu')(input_layer_NO)

## MERGING THE TWO INPUTS
merge_layer = Concatenate()([hidden_layer_1, hidden_layer_2, hidden_layer_3, hidden_layer_CO, hidden_layer_BE, hidden_layer_NO])
## one more layer
hidden_layer = Dense(units = 32, activation = 'relu')(merge_layer)
hidden_layer = Dropout(0.1)(hidden_layer)
# output layers
output_layer = Dense(units = 3)(hidden_layer)

## creating the model
mlp_7 = Model(inputs = [input_layer_1, input_layer_2, input_layer_3, input_layer_CO, input_layer_BE, input_layer_NO], outputs = output_layer)

# compilling the model
mlp_7.compile(optimizer = Adam(learning_rate = 0.0005), loss = 'mse', metrics = [RootMeanSquaredError()])

# instantiating the early stopping callback
es = EarlyStopping(monitor = 'val_root_mean_squared_error', patience = 20, min_delta = 0.002, restore_best_weights = True)

# instantiating the learning rate scheduller
lrs = ReduceLROnPlateau(monitor = 'val_root_mean_squared_error', factor = 0.2, patience = 5)

# fitting the model
history = mlp_7.fit(x = [X_mlp_7_sensor, X_temp_mlp_7, X_rl_mlp_7, X_ar_CO_mlp_7, X_ar_BE_mlp_7, X_ar_NO_mlp_7], 
                    y = y_ar_mlp_7, 
                    validation_split = 0.3, batch_size = 8, epochs = 100, callbacks = [es, lrs])

# training history
plt.figure(figsize = (10, 6))
plt.plot(history.history['loss'], label = 'training')
plt.plot(history.history['val_loss'], label = 'validation')
plt.title(label = 'Training over epochs', fontdict = {'size': 14, 'fontweight': 'bold'})
plt.ylabel(ylabel = 'RMSE', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.xlabel(xlabel = 'Epochs', fontdict = {'size': 12, 'fontweight': 'bold'})
plt.legend(fontsize = 12)
plt.tight_layout()
plt.show()

In [ ]:
## visualizing the model
plot_model(mlp_7, show_shapes = True)

In [ ]:
## seeding the model prediction with the last six observations in the train set
test_seed_CO, test_seed_BE, test_seed_NO = y_CO[-26:-1].tolist(), y_BE[-26:-1].tolist(), y_NO[-26:-1].tolist()

## looping over each of the rows of the test set and generating the predictions
for instance in range(X_test.shape[0]):
    ## parsing the values to a numpy array
    test_ar_CO, test_ar_BE, test_ar_NO = np.array(test_seed_CO[-25:]), np.array(test_seed_BE[-25:]), np.array(test_seed_NO[-25:]), 

    ## extracting the predictions on the test set
    test_preds = mlp_7.predict([
        X_test[instance, sensor_index].reshape(1, X_mlp_7_sensor.shape[1]),
        test_ar_TEMP[instance, :].reshape(1, X_temp_mlp_7.shape[1]),
        test_ar_RL[instance, :].reshape(1, X_rl_mlp_7.shape[1]),
        test_ar_CO.reshape(1, X_ar_CO_mlp_7.shape[1]),
        test_ar_BE.reshape(1, X_ar_BE_mlp_7.shape[1]),
        test_ar_NO.reshape(1, X_ar_NO_mlp_7.shape[1])
    ]
    )
    
    ## reshaping the predictions
    test_preds = np.reshape(test_preds, (-1, ))
    
    ## adding the predictions back to the original lists
    test_seed_CO.append(test_preds[0]), test_seed_BE.append(test_preds[1]), test_seed_NO.append(test_preds[2])
    
    ## printing the progress
    if instance % 200 == 0:
        print(f'Calculating the predictions for the time step {instance}.')

In [ ]:
## stacking the predictions side by side
test_preds = np.column_stack([test_seed_CO, test_seed_BE, test_seed_NO])

## removing the first six observation since they come from the train set
test_preds = test_preds[25:, :]

## putting the test predictions back on the original scale
test_preds = np.expm1(test_preds)

## copying the submission df
submission_mlp_7 = submission.copy()

## putting the predictions on the submission dataset
submission_mlp_7[target_columns] = test_preds

In [ ]:
## predictions
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(range(train.shape[0]), train[target], label = 'Train')
    plt.plot(range(train.shape[0], train.shape[0] + test.shape[0]), submission_mlp_7[target], label = 'Test')
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Value', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.legend()
    plt.tight_layout()
plt.show()

In [ ]:
## getting the predictions of the model
mlp_7_predictions = mlp_7.predict([X_mlp_7_sensor, X_temp_mlp_7, X_rl_mlp_7, X_ar_CO_mlp_7, X_ar_BE_mlp_7, X_ar_NO_mlp_7])

## putting the predictions in a dataframe
mlp_7_predictions = pd.DataFrame(data = mlp_7_predictions, columns = target_columns)

## getting the observed values
observed_values = pd.DataFrame(data = y_ar_mlp_7, columns = target_columns)

## residuals over time
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.plot(observed_values.loc[:, target] - mlp_7_predictions.loc[:, target])
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Timesteps', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Residuals', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# distribution of residuals
plt.figure(figsize = (25, 6))
for idx, target in enumerate(target_columns):
    plt.subplot(1, 3, idx + 1)
    plt.hist(observed_values.loc[:, target] - mlp_7_predictions.loc[:, target], bins = 50)
    plt.title(label = target, fontdict = {'size': 14, 'fontweight': 'bold'})
    plt.xlabel(xlabel = 'Residuals', fontdict = {'size': 14})
    plt.ylabel(ylabel = 'Frequency', fontdict = {'size': 14})
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.tight_layout()
plt.show()

# Saving the submission file

In [ ]:
submission_mlp_1.to_csv(path_or_buf = 'submission_mlp_1.csv', index = False)
submission_mlp_2.to_csv(path_or_buf = 'submission_mlp_2.csv', index = False)
submission_mlp_3.to_csv(path_or_buf = 'submission_mlp_3.csv', index = False)
submission_mlp_4.to_csv(path_or_buf = 'submission_mlp_4.csv', index = False)
submission_mlp_5.to_csv(path_or_buf = 'submission_mlp_5.csv', index = False)
submission_mlp_6.to_csv(path_or_buf = 'submission_mlp_6.csv', index = False)
submission_mlp_7.to_csv(path_or_buf = 'submission_mlp_7.csv', index = False)